In [1]:
import os 
from PIL import Image 
import numpy as np 
import tensorflow as tf 

In [2]:
lr_dir = r"C:/Users/91995/Downloads/div2k/DIV2K_valid_HR/DIV2K_valid_HR"
hr_dir = r"C:/Users/91995/Downloads/div2k/DIV2K_valid_HR/DIV2K_valid_HR"

In [18]:
scale_size = 2
HR_SIZE = (400, 400)
LR_SIZE = (200, 200)
patch_size = 10  
stride = 5

In [19]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
import cv2
def load_images_from_dir(directory, target_size=None):
    images = []
    valid_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}
    
    for filename in os.listdir(directory):
        # Check if file has valid image extension
        if os.path.splitext(filename.lower())[1] in valid_extensions:
            img_path = os.path.join(directory, filename)
            try:
                img = Image.open(img_path).convert('RGB')
                img_array = np.array(img, dtype=np.uint8)
                
                # Ensure image is 3D (height, width, channels)
                if len(img_array.shape) != 3:
                    continue
                    
                img_tensor = tf.convert_to_tensor(img_array, dtype=tf.uint8)
                img_tensor = tf.cast(img_tensor, tf.float32) / 255.0
                
                if target_size:
                    img_tensor = tf.image.resize(img_tensor, target_size, method='area')
                
                img_array = tf.cast(img_tensor * 255.0, tf.uint8).numpy()
                img_yuv = cv2.cvtColor(img_array, cv2.COLOR_RGB2YUV)
                images.append(img_yuv)
            except Exception as e:
                print(f"Error loading {filename}: {str(e)}")
                continue
    
    if not images:
        raise ValueError(f"No valid images found in directory: {directory}")
        
    return np.array(images, dtype=np.uint8)

In [20]:
lr_images = load_images_from_dir(lr_dir, target_size=(200, 200))  
hr_images = load_images_from_dir(hr_dir, target_size=(400, 400)) 

In [21]:
def process_input(input, input_size, upscale_factor):
    input = tf.cast(input, tf.float32) / 255.0
    lr_image = tf.image.resize(input, [input_size, input_size], method="area")
    return lr_image

def process_target(input):
    input = tf.cast(input, tf.float32) / 255.0
    hr_image = tf.image.resize(input, HR_SIZE, method='bilinear')
    return hr_image

def preprocess_image(data):
    hr_image = process_target(data)
    lr_image = process_input(data, LR_SIZE[0], scale_size)
    return lr_image, hr_image

def create_patches(image, patch_size, stride):
    # Ensure input is 4D (batch, height, width, channels)
    if len(image.shape) == 3:
        image = tf.expand_dims(image, 0)
    
    patches = tf.image.extract_patches(
        images=image,
        sizes=[1, patch_size, patch_size, 1],
        strides=[1, stride, stride, 1],
        rates=[1, 1, 1, 1],
        padding='VALID'
    )
    
    # Reshape patches to (num_patches, patch_size, patch_size, channels)
    patches = tf.reshape(patches, [-1, patch_size, patch_size, image.shape[-1]])
    return patches

def apply_preprocessing_on_local_data(lr_images, hr_images):
    if len(lr_images) != len(hr_images):
        raise ValueError("Number of LR and HR images must match")
    
    lr_patches_list = []
    hr_patches_list = []
    
    for i, (lr_image, hr_image) in enumerate(zip(lr_images, hr_images)):
        try:
            # Preprocess images
            lr_processed, hr_processed = preprocess_image(hr_image)
            
            # Create patches
            lr_patches = create_patches(lr_processed, patch_size, stride)
            hr_patches = create_patches(hr_processed, patch_size * scale_size, stride * scale_size)
            
            # Verify patches were created successfully
            if tf.shape(lr_patches)[0] > 0 and tf.shape(hr_patches)[0] > 0:
                lr_patches_list.append(lr_patches)
                hr_patches_list.append(hr_patches)
            else:
                print(f"Warning: No patches created for image {i}")
                
        except Exception as e:
            print(f"Error processing image {i}: {str(e)}")
            continue
    
    if not lr_patches_list or not hr_patches_list:
        raise ValueError("No valid patches were created from any images")
    
    # Concatenate all patches
    lr_patches = tf.concat(lr_patches_list, axis=0)
    hr_patches = tf.concat(hr_patches_list, axis=0)
    
    return lr_patches, hr_patches


In [22]:
try:
    # Load images
    lr_images = load_images_from_dir(lr_dir, target_size=LR_SIZE)
    hr_images = load_images_from_dir(hr_dir, target_size=HR_SIZE)
    
    print(f"Loaded {len(lr_images)} LR images and {len(hr_images)} HR images")
    
    # Create patches
    lr_patches, hr_patches = apply_preprocessing_on_local_data(lr_images, hr_images)
    
    print(f"Created {tf.shape(lr_patches)[0]} LR patches and {tf.shape(hr_patches)[0]} HR patches")
    
except Exception as e:
    print(f"Error in main processing: {str(e)}")

Loaded 100 LR images and 100 HR images
Created 152100 LR patches and 152100 HR patches


In [23]:
lr_patches, hr_patches = apply_preprocessing_on_local_data(lr_images, hr_images)
#print("Low-res patches shape:", lr_patches.shape)
#print("High-res patches shape:", hr_patches.shape)

In [24]:
def psnr(y_true, y_pred):
    max_pixel = 1.0
    return tf.image.psnr(y_true, y_pred, max_val=max_pixel)

class DepthToSpace(tf.keras.layers.Layer):
    def __init__(self, scale, **kwargs):
        super(DepthToSpace, self).__init__(**kwargs)
        self.scale = scale

    def call(self, inputs):
        return tf.nn.depth_to_space(inputs, self.scale)

def leaky_relu_activation(x):
    leaky_relu_out = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
    linear_out = x
    return leaky_relu_out + linear_out

class SinglePixelAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(SinglePixelAttention, self).__init__(**kwargs)
        
    def call(self, inputs):
        mean_values = tf.reduce_mean(inputs, axis=-1)
        max_indices = tf.argmax(mean_values, axis=-1)
        attention_mask = tf.one_hot(max_indices, depth=tf.shape(inputs)[1])
        attention_mask = tf.expand_dims(attention_mask, axis=-1)
        attention_output = inputs * attention_mask
        return attention_output

In [25]:
def build_espcn_model_with_skip_connections(input_shape, scale_size):
    inputs = tf.keras.layers.Input(shape=input_shape)
    conv1 = tf.keras.layers.Conv2D(16, 5, padding='same', kernel_initializer=tf.keras.initializers.HeNormal())(inputs)
    conv1 = tf.keras.layers.LeakyReLU(alpha=0.2)(conv1)
    conv2 = tf.keras.layers.Conv2D(16, 3, padding='same', kernel_initializer=tf.keras.initializers.HeNormal())(conv1)
    conv2 = tf.keras.layers.LeakyReLU(alpha=0.2)(conv2)
    skip1 = tf.keras.layers.Add()([conv1, conv2])
    conv3 = tf.keras.layers.Conv2D(16, 2, padding='same', kernel_initializer='orthogonal')(skip1)
    conv3 = tf.keras.layers.Lambda(leaky_relu_activation)(conv3)
    conv3 = SinglePixelAttention()(conv3)
    skip2 = tf.keras.layers.Add()([skip1, conv3])
    conv4 = tf.keras.layers.Conv2D(3 * (scale_size ** 2), 3, padding='same', kernel_initializer='orthogonal')(skip2)
    conv4 = tf.keras.layers.LeakyReLU(alpha=0.2)(conv4)
    outputs = DepthToSpace(scale_size)(conv4)
    model = tf.keras.models.Model(inputs, outputs)
    return model

In [26]:
input_shape = (patch_size, patch_size, 3)
espcn_model_with_skip_and_attention = build_espcn_model_with_skip_connections(input_shape, scale_size)
espcn_model_with_skip_and_attention.compile(optimizer='adam', loss='mse', metrics=[psnr])

In [27]:
espcn_model_with_skip_and_attention.fit(lr_patches, hr_patches, epochs=20)

Epoch 1/20
4754/4754 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 0.0077 - psnr: 27.3023
Epoch 2/20
4754/4754 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - loss: 8.0837e-04 - psnr: 34.3721
Epoch 3/20
4754/4754 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - loss: 7.5276e-04 - psnr: 35.1159
Epoch 4/20
4754/4754 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 7.3351e-04 - psnr: 35.4071
Epoch 5/20
4754/4754 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - loss: 7.2575e-04 - psnr: 35.5280
Epoch 6/20
4754/4754 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - loss: 7.1931e-04 - psnr: 35.6660
Epoch 7/20
4754/4754 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - loss: 7.1558e-04 - psnr: 35.7579
Epoch 8/20
4754/4754 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - loss: 7.1535e-04 - psnr: 35.8222
Epoch 9/20
4754/4754 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - loss: 7.1441e-04 - psnr: 35.8643
Epoch 10/20
4754/4754 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 7.0926e-04 - psnr: 35.9473
Epoch 11/20
4754/4754 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - loss: 7.1049e-04 - psnr: 35.9013
Epoch 12/20


In [37]:
input_file = r"C:/Users/91995/Downloads/Set5/Set5/image_SRF_2/img_005_SRF_2_LR.png"
output_file = r"C:/Users/91995/Downloads/Set5/Set5/image_SRF_2/Gal"
gt = r"C:/Users/91995/Downloads/Set5/Set5/image_SRF_2/img_005_SRF_2_HR.png"



In [34]:
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
import os

class ESPCNInference:
    def __init__(self, model_path=None, scale_size=2, patch_size=10):
        self.scale_size = scale_size  
        self.patch_size = patch_size
        self.model = None
        if model_path:
            self.load_model(model_path)
        else:

            input_shape = (patch_size, patch_size, 3)
            self.model = build_espcn_model_with_skip_connections(input_shape, scale_size)
    
    def load_model(self, model_path):

        try:
            self.model = tf.keras.models.load_model(
                model_path,
                custom_objects={
                    'DepthToSpace': DepthToSpace,
                    'SinglePixelAttention': SinglePixelAttention,
                    'psnr': psnr
                }
            )
            print("Model loaded successfully")
        except Exception as e:
            print(f"Error loading model: {str(e)}")
            raise
    
    def save_model(self, model_path):

        self.model.save(model_path)
        print(f"Model saved to {model_path}")
    
    def preprocess_image(self, image_path):
        try:
            
            img = Image.open(image_path).convert('RGB')
            img_array = np.array(img, dtype=np.uint8)
            img_yuv = cv2.cvtColor(img_array, cv2.COLOR_RGB2YUV)
            
            #
            img_yuv = img_yuv.astype(np.float32) / 255.0
            
            
            img_yuv = np.expand_dims(img_yuv, axis=0)
            
            return img_yuv, img_array.shape[:2]
            
        except Exception as e:
            print(f"Error preprocessing image: {str(e)}")
            raise
    
    def create_patches(self, image, include_padding=True):

        h, w = image.shape[1:3]
        
        if include_padding:
            
            pad_h = (self.patch_size - h % self.patch_size) % self.patch_size
            pad_w = (self.patch_size - w % self.patch_size) % self.patch_size
            
            
            if pad_h > 0 or pad_w > 0:
                image = tf.pad(image, [[0, 0], [0, pad_h], [0, pad_w], [0, 0]], mode='REFLECT')
        
        
        patches = tf.image.extract_patches(
            images=image,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding='VALID'
        )
        
        
        patches = tf.reshape(patches, [-1, self.patch_size, self.patch_size, 3])
        return patches, image.shape[1:3]
    
    def reconstruct_image(self, patches, original_size, padded_size):

        
        h, w = padded_size
        patch_size_hr = self.patch_size * self.scale_size  
        patches_per_row = w // self.patch_size
        patches_per_col = h // self.patch_size
        
        
        patches = tf.reshape(patches, [patches_per_col, patches_per_row, 
                                     patch_size_hr, patch_size_hr, 3])
        
        
        image = tf.transpose(patches, [0, 2, 1, 3, 4])
        image = tf.reshape(image, [1, h * self.scale_size, w * self.scale_size, 3])
        
        
        orig_h, orig_w = original_size
        image = image[:, :orig_h * self.scale_size, :orig_w * self.scale_size, :]
        
        return image
    
    def postprocess_image(self, image):

        image = tf.clip_by_value(image, 0, 1)
        image = tf.round(image * 255.0)
        image = tf.cast(image, tf.uint8)
        image = cv2.cvtColor(image[0].numpy(), cv2.COLOR_YUV2RGB)
        return image
    
    def upscale_image(self, image_path, output_path=None):
      
        try:

            img_yuv, original_size = self.preprocess_image(image_path)
            
            patches, padded_size = self.create_patches(img_yuv)
            
           
            sr_patches = self.model.predict(patches, verbose=0)
            
           
            sr_image = self.reconstruct_image(sr_patches, original_size, padded_size)
            
           
            final_image = self.postprocess_image(sr_image)
            
            if output_path:
                Image.fromarray(final_image).save(output_path)
                print(f"Saved super-resolved image to {output_path}")
            
            return final_image
            
        except Exception as e:
            print(f"Error during upscaling: {str(e)}")
            raise
    
    def upscale_directory(self, input_dir, output_dir):

        os.makedirs(output_dir, exist_ok=True)
        
        for filename in os.listdir(input_dir):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                input_path = os.path.join(input_dir, filename)
                output_path = os.path.join(output_dir, f"sr_{filename}")
                
                try:
                    self.upscale_image(input_path, output_path)
                    print(f"Processed {filename}")
                except Exception as e:
                    print(f"Error processing {filename}: {str(e)}")
                    continue

# Example usage:
"""
# After training your model:
inference_model = ESPCNInference(scale_size=2, patch_size=10)  # Changed scale_size to 2

# Transfer weights from trained model
inference_model.model.set_weights(espcn_model_with_skip_and_attention.get_weights())

# Save the model if needed
inference_model.save_model('espcn_model.h5')

# Upscale a single image
upscaled_image = inference_model.upscale_image('input.jpg', 'output.jpg')

# Or upscale all images in a directory
inference_model.upscale_directory('input_dir', 'output_dir')
"""

"\n# After training your model:\ninference_model = ESPCNInference(scale_size=2, patch_size=10)  # Changed scale_size to 2\n\n# Transfer weights from trained model\ninference_model.model.set_weights(espcn_model_with_skip_and_attention.get_weights())\n\n# Save the model if needed\ninference_model.save_model('espcn_model.h5')\n\n# Upscale a single image\nupscaled_image = inference_model.upscale_image('input.jpg', 'output.jpg')\n\n# Or upscale all images in a directory\ninference_model.upscale_directory('input_dir', 'output_dir')\n"

In [35]:
inference_model = ESPCNInference(scale_size=4, patch_size=10)
upscaled_image = inference_model.upscale_image(input_file, output_file)

Error during upscaling: unknown file extension: 


ValueError: unknown file extension: 

In [36]:
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
import os

class ESPCNInference:
    def __init__(self, model_path=None, scale_size=2, patch_size=10):
        self.scale_size = scale_size
        self.patch_size = patch_size
        self.model = None
        if model_path:
            self.load_model(model_path)
        else:
            input_shape = (patch_size, patch_size, 3)
            self.model = build_espcn_model_with_skip_connections(input_shape, scale_size)
    
    def load_model(self, model_path):
        """Load trained model weights"""
        try:
            self.model = tf.keras.models.load_model(
                model_path,
                custom_objects={
                    'DepthToSpace': DepthToSpace,
                    'SinglePixelAttention': SinglePixelAttention,
                    'psnr': psnr
                }
            )
            print("Model loaded successfully")
        except Exception as e:
            print(f"Error loading model: {str(e)}")
            raise
    
    def save_model(self, model_path):
        """Save the trained model"""
        self.model.save(model_path)
        print(f"Model saved to {model_path}")
    
    def calculate_psnr(self, img1, img2):
        """Calculate PSNR between two images"""
        # Convert to float32
        img1 = img1.astype(np.float32)
        img2 = img2.astype(np.float32)
        
        # Calculate MSE
        mse = np.mean((img1 - img2) ** 2)
        if mse == 0:
            return float('inf')
        
        # Calculate PSNR
        max_pixel = 255.0
        psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
        return psnr
    
    def calculate_ssim(self, img1, img2):
        """Calculate SSIM between two images"""
        C1 = (0.01 * 255)**2
        C2 = (0.03 * 255)**2
        
        img1 = img1.astype(np.float32)
        img2 = img2.astype(np.float32)
        
        kernel = cv2.getGaussianKernel(11, 1.5)
        window = np.outer(kernel, kernel.transpose())
        
        mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]
        mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
        mu1_sq = mu1**2
        mu2_sq = mu2**2
        mu1_mu2 = mu1 * mu2
        
        sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
        sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
        sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2
        
        ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / \
                   ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
        return np.mean(ssim_map)
    
    def evaluate_image_quality(self, sr_image, hr_image):
        """Calculate quality metrics between super-resolved and high-resolution images"""
        # Ensure images are the same size
        if sr_image.shape != hr_image.shape:
            hr_image = cv2.resize(hr_image, (sr_image.shape[1], sr_image.shape[0]))
        
        # Calculate metrics for each channel
        psnr_values = []
        ssim_values = []
        
        for c in range(3):  # For each RGB channel
            psnr_values.append(self.calculate_psnr(sr_image[:,:,c], hr_image[:,:,c]))
            ssim_values.append(self.calculate_ssim(sr_image[:,:,c], hr_image[:,:,c]))
        
        metrics = {
            'psnr': {
                'average': np.mean(psnr_values),
                'per_channel': psnr_values
            },
            'ssim': {
                'average': np.mean(ssim_values),
                'per_channel': ssim_values
            }
        }
        
        return metrics
    
    def preprocess_image(self, image_path):
        """Preprocess a single image for inference"""
        try:
            img = Image.open(image_path).convert('RGB')
            img_array = np.array(img, dtype=np.uint8)
            img_yuv = cv2.cvtColor(img_array, cv2.COLOR_RGB2YUV)
            img_yuv = img_yuv.astype(np.float32) / 255.0
            img_yuv = np.expand_dims(img_yuv, axis=0)
            return img_yuv, img_array.shape[:2]
        except Exception as e:
            print(f"Error preprocessing image: {str(e)}")
            raise
    
    def create_patches(self, image, include_padding=True):
        """Create patches from the input image with optional padding"""
        h, w = image.shape[1:3]
        
        if include_padding:
            pad_h = (self.patch_size - h % self.patch_size) % self.patch_size
            pad_w = (self.patch_size - w % self.patch_size) % self.patch_size
            
            if pad_h > 0 or pad_w > 0:
                image = tf.pad(image, [[0, 0], [0, pad_h], [0, pad_w], [0, 0]], mode='REFLECT')
        
        patches = tf.image.extract_patches(
            images=image,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding='VALID'
        )
        
        patches = tf.reshape(patches, [-1, self.patch_size, self.patch_size, 3])
        return patches, image.shape[1:3]
    
    def reconstruct_image(self, patches, original_size, padded_size):
        """Reconstruct the full image from patches"""
        h, w = padded_size
        patch_size_hr = self.patch_size * self.scale_size
        patches_per_row = w // self.patch_size
        patches_per_col = h // self.patch_size
        
        patches = tf.reshape(patches, [patches_per_col, patches_per_row, 
                                     patch_size_hr, patch_size_hr, 3])
        
        image = tf.transpose(patches, [0, 2, 1, 3, 4])
        image = tf.reshape(image, [1, h * self.scale_size, w * self.scale_size, 3])
        
        orig_h, orig_w = original_size
        image = image[:, :orig_h * self.scale_size, :orig_w * self.scale_size, :]
        
        return image
    
    def postprocess_image(self, image):
        """Convert image back to RGB and uint8 format"""
        image = tf.clip_by_value(image, 0, 1)
        image = tf.round(image * 255.0)
        image = tf.cast(image, tf.uint8)
        image = cv2.cvtColor(image[0].numpy(), cv2.COLOR_YUV2RGB)
        return image
    
    def upscale_image(self, image_path, output_path=None, hr_image_path=None):
        """Upscale a single image and optionally calculate quality metrics"""
        try:
            # Preprocess
            img_yuv, original_size = self.preprocess_image(image_path)
            
            # Create patches
            patches, padded_size = self.create_patches(img_yuv)
            
            # Process patches
            sr_patches = self.model.predict(patches, verbose=0)
            
            # Reconstruct image
            sr_image = self.reconstruct_image(sr_patches, original_size, padded_size)
            
            # Postprocess
            final_image = self.postprocess_image(sr_image)
            
            # Calculate metrics if HR image is provided
            metrics = None
            if hr_image_path:
                hr_image = np.array(Image.open(hr_image_path).convert('RGB'))
                metrics = self.evaluate_image_quality(final_image, hr_image)
                print("\nImage Quality Metrics:")
                print(f"PSNR: {metrics['psnr']['average']:.2f} dB")
                print(f"SSIM: {metrics['ssim']['average']:.4f}")
            
            if output_path:
                Image.fromarray(final_image).save(output_path)
                print(f"Saved super-resolved image to {output_path}")
            
            return final_image, metrics
            
        except Exception as e:
            print(f"Error during upscaling: {str(e)}")
            raise
    
    def _ensure_valid_extension(self, filepath):
        """Ensure the filepath has a valid image extension"""
        valid_extensions = {'.png', '.jpg', '.jpeg', '.bmp', '.tiff'}
        _, ext = os.path.splitext(filepath)
        ext = ext.lower()
        
        if not ext:
            # If no extension, default to PNG
            filepath = f"{filepath}.png"
        elif ext not in valid_extensions:
            # If invalid extension, append PNG
            filepath = f"{filepath}.png"
            
        return filepath

    def upscale_image(self, image_path, output_path=None, hr_image_path=None):
        """Upscale a single image and optionally calculate quality metrics"""
        try:
            # Preprocess
            img_yuv, original_size = self.preprocess_image(image_path)
            
            # Create patches
            patches, padded_size = self.create_patches(img_yuv)
            
            # Process patches
            sr_patches = self.model.predict(patches, verbose=0)
            
            # Reconstruct image
            sr_image = self.reconstruct_image(sr_patches, original_size, padded_size)
            
            # Postprocess
            final_image = self.postprocess_image(sr_image)
            
            # Calculate metrics if HR image is provided
            metrics = None
            if hr_image_path:
                hr_image = np.array(Image.open(hr_image_path).convert('RGB'))
                metrics = self.evaluate_image_quality(final_image, hr_image)
                print("\nImage Quality Metrics:")
                print(f"PSNR: {metrics['psnr']['average']:.2f} dB")
                print(f"SSIM: {metrics['ssim']['average']:.4f}")
            
            if output_path:
                # Ensure valid file extension
                output_path = self._ensure_valid_extension(output_path)
                # Convert numpy array to PIL Image and save
                Image.fromarray(final_image).save(output_path)
                print(f"Saved super-resolved image to {output_path}")
            
            return final_image, metrics
            
        except Exception as e:
            print(f"Error during upscaling: {str(e)}")
            raise
    
    def _ensure_valid_extension(self, filepath):
        """Ensure the filepath has a valid image extension"""
        valid_extensions = {'.png', '.jpg', '.jpeg', '.bmp', '.tiff'}
        _, ext = os.path.splitext(filepath)
        ext = ext.lower()
        
        if not ext:
            # If no extension, default to PNG
            filepath = f"{filepath}.png"
        elif ext not in valid_extensions:
            # If invalid extension, append PNG
            filepath = f"{filepath}.png"
            
        return filepath

    def upscale_image(self, image_path, output_path=None, hr_image_path=None):
        """Upscale a single image and optionally calculate quality metrics"""
        try:
            # Preprocess
            img_yuv, original_size = self.preprocess_image(image_path)
            
            # Create patches
            patches, padded_size = self.create_patches(img_yuv)
            
            # Process patches
            sr_patches = self.model.predict(patches, verbose=0)
            
            # Reconstruct image
            sr_image = self.reconstruct_image(sr_patches, original_size, padded_size)
            
            # Postprocess
            final_image = self.postprocess_image(sr_image)
            
            # Calculate metrics if HR image is provided
            metrics = None
            if hr_image_path:
                hr_image = np.array(Image.open(hr_image_path).convert('RGB'))
                metrics = self.evaluate_image_quality(final_image, hr_image)
                print("\nImage Quality Metrics:")
                print(f"PSNR: {metrics['psnr']['average']:.2f} dB")
                print(f"SSIM: {metrics['ssim']['average']:.4f}")
            
            if output_path:
                # Ensure valid file extension
                output_path = self._ensure_valid_extension(output_path)
                # Convert numpy array to PIL Image and save
                Image.fromarray(final_image).save(output_path)
                print(f"Saved super-resolved image to {output_path}")
            
            return final_image, metrics
            
        except Exception as e:
            print(f"Error during upscaling: {str(e)}")
            raise
    
    def upscale_directory(self, input_dir, output_dir, hr_dir=None):
        """Upscale all images in a directory and calculate metrics if HR images are available"""
        os.makedirs(output_dir, exist_ok=True)
        
        metrics_summary = []
        
        for filename in os.listdir(input_dir):
            base, ext = os.path.splitext(filename)
            if ext.lower() in {'.png', '.jpg', '.jpeg', '.bmp', '.tiff'}:
                input_path = os.path.join(input_dir, filename)
                output_path = os.path.join(output_dir, f"sr_{base}.png")  # Always save as PNG
                
                # Check for corresponding HR image
                hr_path = None
                if hr_dir:
                    # Look for HR image with any valid extension
                    hr_base = os.path.join(hr_dir, base)
                    for hr_ext in ['.png', '.jpg', '.jpeg', '.bmp', '.tiff']:
                        potential_hr_path = hr_base + hr_ext
                        if os.path.exists(potential_hr_path):
                            hr_path = potential_hr_path
                            break
                    
                    if not hr_path:
                        print(f"Warning: No HR image found for {filename}")
                
                try:
                    _, metrics = self.upscale_image(input_path, output_path, hr_path)
                    if metrics:
                        metrics_summary.append({
                            'filename': filename,
                            'psnr': metrics['psnr']['average'],
                            'ssim': metrics['ssim']['average']
                        })
                    print(f"Processed {filename}")
                except Exception as e:
                    print(f"Error processing {filename}: {str(e)}")
                    continue
        
        # Calculate and print average metrics
        if metrics_summary:
            avg_psnr = np.mean([m['psnr'] for m in metrics_summary])
            avg_ssim = np.mean([m['ssim'] for m in metrics_summary])
            print("\nDirectory Summary:")
            print(f"Average PSNR: {avg_psnr:.2f} dB")
            print(f"Average SSIM: {avg_ssim:.4f}")
            
            # Save metrics to CSV
            metrics_path = os.path.join(output_dir, 'metrics_summary.csv')
            with open(metrics_path, 'w') as f:
                f.write('filename,psnr,ssim\n')
                for m in metrics_summary:
                    f.write(f"{m['filename']},{m['psnr']:.2f},{m['ssim']:.4f}\n")
            print(f"\nMetrics summary saved to {metrics_path}")

# Example usage:
"""
# Initialize model
inference_model = ESPCNInference(scale_size=2, patch_size=10)

# Load trained weights
inference_model.load_model('espcn_model.h5')

# Upscale a single image and calculate metrics
upscaled_image, metrics = inference_model.upscale_image(
    'input.jpg',
    'output',  # Will automatically add .png extension
    'ground_truth_hr.jpg'  # Optional HR image for quality assessment
)

# Upscale all images in a directory and calculate metrics
inference_model.upscale_directory(
    'input_dir',
    'output_dir',
    'hr_dir'  # Optional directory containing HR images
)
"""

"\n# Initialize model\ninference_model = ESPCNInference(scale_size=2, patch_size=10)\n\n# Load trained weights\ninference_model.load_model('espcn_model.h5')\n\n# Upscale a single image and calculate metrics\nupscaled_image, metrics = inference_model.upscale_image(\n    'input.jpg',\n    'output',  # Will automatically add .png extension\n    'ground_truth_hr.jpg'  # Optional HR image for quality assessment\n)\n\n# Upscale all images in a directory and calculate metrics\ninference_model.upscale_directory(\n    'input_dir',\n    'output_dir',\n    'hr_dir'  # Optional directory containing HR images\n)\n"

In [38]:
inference_model = ESPCNInference(scale_size=2, patch_size=10)
upscaled_image, metrics = inference_model.upscale_image(
    input_file,
    output_file,  # Will automatically add .png extension
    gt  # Optional HR image for quality assessment
)

c:\Users\91995\anaconda3\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(



Image Quality Metrics:
PSNR: 5.94 dB
SSIM: 0.0109
Saved super-resolved image to C:/Users/91995/Downloads/Set5/Set5/image_SRF_2/Gal.png
